<a href="https://colab.research.google.com/github/nikhil6553/studious-waffle/blob/main/synthetic_data_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
pip install sdv pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.0/183.0 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.8/73.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.1/198.1 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
import seaborn as sns
import pandas as pd
from ctgan import CTGAN
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Load Titanic dataset
df = sns.load_dataset('titanic')

# Drop irrelevant or high-missing columns
df = df.drop(columns=['deck', 'embark_town', 'alive', 'class', 'who'])

# Drop rows with missing values
df = df.dropna()

# Convert target to binary
df['survived'] = df['survived'].astype('category')

# Identify discrete (categorical) columns
discrete_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()

# Print preprocessed info
print("📊 Cleaned Titanic Dataset Shape:", df.shape)
print("🧩 Discrete Columns:", discrete_columns)

# Train CTGAN model
ctgan = CTGAN(epochs=100)
ctgan.fit(df, discrete_columns=discrete_columns)

# Save CTGAN model
with open("ctgan_titanic_model.pkl", "wb") as f:
    pickle.dump(ctgan, f)
print("✅ CTGAN model saved as 'ctgan_titanic_model.pkl'")

# Generate synthetic data
synthetic_data = ctgan.sample(len(df))

# Encode both datasets for ML
real_encoded = pd.get_dummies(df, drop_first=True)
synthetic_encoded = pd.get_dummies(synthetic_data, drop_first=True)

# Align both encoded dataframes
real_encoded, synthetic_encoded = real_encoded.align(synthetic_encoded, join='inner', axis=1)

# Split into features and label
X_real = real_encoded.drop('survived_1', axis=1)
y_real = real_encoded['survived_1']

X_synth = synthetic_encoded.drop('survived_1', axis=1)
y_synth = synthetic_encoded['survived_1']

# Train classifier on synthetic data, test on real data
clf = RandomForestClassifier()
clf.fit(X_synth, y_synth)
y_pred = clf.predict(X_real)

# Evaluation
print("\n🎯 Evaluation on Real Data (trained on synthetic data):\n")
print(classification_report(y_real, y_pred))


📊 Cleaned Titanic Dataset Shape: (712, 10)
🧩 Discrete Columns: ['survived', 'sex', 'embarked']


/usr/local/lib/python3.11/dist-packages/torch/autograd/graph.py:823: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at /pytorch/aten/src/ATen/cuda/CublasHandlePool.cpp:180.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


✅ CTGAN model saved as 'ctgan_titanic_model.pkl'

🎯 Evaluation on Real Data (trained on synthetic data):

              precision    recall  f1-score   support

       False       0.54      0.53      0.54       424
        True       0.33      0.34      0.34       288

    accuracy                           0.46       712
   macro avg       0.44      0.44      0.44       712
weighted avg       0.46      0.46      0.46       712



In [15]:
import pickle

# Load the saved CTGAN model
with open("ctgan_titanic_model.pkl", "rb") as f:
    ctgan = pickle.load(f)

# Generate synthetic data (e.g., 500 rows)
synthetic_data = ctgan.sample(500)

# Display the synthetic data
print("🧪 Synthetic Titanic Data:")
print(synthetic_data.head())

# save to CSV
synthetic_data.to_csv("synthetic_titanic_data.csv", index=False)
print("✅ Synthetic data saved to 'synthetic_titanic_data.csv'")


🧪 Synthetic Titanic Data:
  survived  pclass     sex        age  sibsp  parch        fare embarked  \
0        0       3    male  15.703742      0      0   24.548846        C   
1        0       3    male  45.404523      1      1  117.743174        S   
2        1       1  female  17.679916      0      1    9.639219        C   
3        0       3    male  16.623379      1      0   52.572573        S   
4        1       2    male  48.450672      1      0   16.391665        C   

   adult_male  alone  
0        True   True  
1        True   True  
2        True   True  
3        True   True  
4        True   True  
✅ Synthetic data saved to 'synthetic_titanic_data.csv'
